In [3]:
import os
import requests
from bs4 import BeautifulSoup
import json
import traceback
import pandas as pd

## Leer archivo estaciones

In [8]:
est = requests.get('https://raw.githubusercontent.com/PeHMeteoN/ScrappingToolKit/master/json/PE_senamhi_current.json')

## Agregando altura

In [12]:
def agregar_altura(cod, estado, tipo_esta, cate, cod_old=None):
    if estado == "AUTOMATICA":
        url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod={}&estado={}&tipo_esta={}&cate={}".format(cod, estado, tipo_esta, cate)
    else:
        url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod={}&estado={}&tipo_esta={}&cate={}&cod_old={}".format(cod, estado, tipo_esta, cate, cod_old)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    for s in soup.find_all("td"):
        if len(s) > 0:
            if "msnm" in s.text:
                alt = s.text.split(" ")[0]
    return alt

Extraer alturas para todos los datos y guardarlo en diccionario

In [32]:
a = 0
for e in est:
    print(a)
    if "cod_old" in e.keys():
        alt = agregar_altura(e["cod"], e["estado"], e["ico"], e["cate"], e["cod_old"])
    else:
        alt = agregar_altura(e["cod"], e["estado"], e["ico"], e["cate"])
    est[a]["alt"] = alt
    a += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

## Crear función para extraer data en tiempo real

In [2]:
def data_senamhi_realtime(estacion, year_month):
    cod = e["cod"]
    tipo_esta = e["ico"]
    estado = e["estado"]
    cod_old = e["cod_old"] if "cod_old" in e.keys() else ""
    cate_esta = e["cate"]
    altitud = e["alt"]
    
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/_dato_esta_tipo02.php"
    new_url = "{}?estaciones={}&CBOFiltro={}&t_e={}&estado={}&cod_old={}&cate_esta={}&alt={}".format(
            url, cod, year_month, tipo_esta, estado, cod_old, cate_esta, altitud)
    
    s = requests.get(new_url)
    html = s.text
    soup = BeautifulSoup(html, 'html.parser')
    tables = [
        [
            [td.get_text(strip=True) for td in tr.find_all('td')] 
            for tr in table.find_all('tr')
        ] 
        for table in soup.find_all('table')
    ]
    
    if tipo_esta == "M":
        if estado == "AUTOMATICA":
            cols = ["fecha", "hora", "temp", "pp", "humedad", "dir_viento", "vel_viento"]
            df = pd.DataFrame(tables[1][1:], columns=cols)
        else:
            cols = ["fecha", "temp_max", "temp_min", "hum_relativa", "pp"]
            df = pd.DataFrame(tables[1][2:], columns=cols)
    elif tipo_esta == "H":
        if estado == "AUTOMATICA":
            cols = ["fecha", "hora", "nivel", "pp"]
            df = pd.DataFrame(tables[1][1:], columns=cols)
        else:
            cols = ["fecha", "nivel_06", "nivel_10", "nivel_14", "nivel_18"]
            df = pd.DataFrame(tables[1][2:], columns=cols)
    return df

In [3]:
def main(estacion, year_month):
    ldates = [str(y) + str(x).zfill(2) for x in range(1,13) for y in range(year_month[0], year_month[1])]
    ldates.sort()
    data = []
    for l in ldates:
        try:
            df = data_senamhi_realtime(estacion, l)
            if type(data) == list:
                data = pd.DataFrame({}, columns = list(df.columns))
            else:
                data = data.append(df)
        except:
            pass
    return data

In [5]:
estaciones = requests.get('https://raw.githubusercontent.com/PeHMeteoN/ScrappingToolKit/master/json/PE_senamhi_current.json')

In [5]:
folder = r'E:\2019\SENAMHI\senamhi_data\data_realtime'
list_error = []
zz = 0
for e in estaciones[900:]:
    print("{}: {} - {}".format(zz, e["ico"], e["nom"]))
    try:
        data = main(e, [1980, 2020])
        path_csv = "{}_{}.csv".format(e["ico"], e["cod"])
        data.to_csv(os.path.join(folder, path_csv))
    except:
        list_error.append(e)
    zz += 1

0: M - SALPO
1: M - HUAMACHUCO
2: M - CACHICADAN
3: M - REQUE
4: M - OYOTUN
5: M - SIPAN
6: M - CAYALTI
7: M - LAMBAYEQUE
8: M - JAYANCA (LA VIÃ‘A)
9: M - PASABAR
10: M - CAMPO DE MARTE
11: M - CHOSICA
12: H - CHOSICA
13: M - Ã‘AÃ‘A
14: H - PUENTE HUACHIPA
15: H - YONAN GORE
16: M - CUTERVO GORE
17: H - TUNEL CHOTANO
18: H - CUMBA
19: M - BAMBAMARCA GORE
20: H - BAMBAMARCA H GORE
21: M - CHUGUR
22: M - JAEN GORE
23: H - HUALLAPE
24: M - SAN IGNACIO GORE
25: M - SAN MARCOS
26: H - TONGOD
27: H - CIRATO CAÃ‘AD
28: H - CIRATO
29: H - PUENTE AMBAN
30: M - CHANCAY BAÃ‘OS
31: M - SORAYPAMPA
32: M - CALCA
33: M - NUEVO PISAC
34: M - SAUSAYA
35: M - PAYAPUNKU
36: M - SICUANI
37: H - SALCCA
38: M - SIBINACOCHA
39: M - QUISOQUIPINA
40: M - MARCA CUNKA
41: M - SANTO TOMAS
42: M - HUAYHUAHUASI
43: M - QUILLABAMBA
44: M - PONGO DE MAINIQUE GORE
45: M - SAN PABLO
46: M - PICHARI
47: H - PAUCARTAMBO
48: M - ACJANACO GORE
49: M - ASHANAKU
50: M - MARCAPATA GORE
51: H - CHILCA
52: M - QORIHUAYRACHINA
5

In [8]:
list_error

[{'nom': 'MAGUNCHAL',
  'cate': 'PLU',
  'lat': -5.89106,
  'lon': -78.18886,
  'ico': 'M',
  'cod': '105046',
  'cod_old': '152210',
  'estado': 'DIFERIDO',
  'alt': '632'},
 {'nom': 'SANTIAGO ANTUNEZ DE MAYOLO',
  'cate': 'MAP',
  'lat': -9.51653,
  'lon': -77.52486,
  'ico': 'M',
  'cod': '109009',
  'cod_old': '000426',
  'estado': 'DIFERIDO',
  'alt': '3079'},
 {'nom': 'CHACCHAN',
  'cate': 'PLU',
  'lat': -9.53519,
  'lon': -77.77536,
  'ico': 'M',
  'cod': '109045',
  'cod_old': '154107',
  'estado': 'DIFERIDO',
  'alt': '2266'}]